In [3]:
path = "/home/tomasruiz/datasets/dss_home/toxicainment/simple_inference_benchmark_results/qwen-2.5-vl/model_labels.csv"

In [ ]:
import pandas as pd


wide = pd.read_csv(path)
wide

In [ ]:
cols = [
    "is_eudaimonic_entertainment",
    "is_hedonic_entertainment",
    "is_intolerant",
    "is_political",
    "is_saxony",
]
comments_cols = [f"{col}_comment" for col in cols]
long = pd.melt(wide, id_vars=["Model ID", "post_id"], value_vars=comments_cols)
long

In [23]:
gt = "/home/tomasruiz/datasets/dss_home/toxicainment/simple_inference_benchmark_results/gemini-2.5-pro-noschema/model_labels.csv"
gt = pd.read_csv(gt)
gt_long = pd.melt(gt, id_vars=["Model ID", "post_id"], value_vars=comments_cols)

In [ ]:
joined = pd.merge(long, gt_long, on=["post_id", "variable"], suffixes=("_ai", "_gt"))

joined

In [ ]:
from bench_lib.evaluation import calc_bertscore
import numpy as np


def func(df):
    row = df.iloc[0]
    gt = row["value_gt"]
    ai = row["value_ai"]
    return calc_bertscore([ai], [gt])
    # return np.random.rand()


joined.groupby(["Model ID_ai", "post_id", "variable"]).apply(func)

In [ ]:
joined.dropna(subset=["value_ai"], inplace=True)

In [ ]:
from bert_score import score


results = score(
    joined["value_ai"].astype("string").tolist(),
    joined["value_gt"].astype("string").tolist(),
    lang="en",
    verbose=False,
    device="cuda",
)

In [ ]:
P, R, F1 = results

In [52]:
joined = joined.assign(bertscore_f1=F1.numpy())

In [ ]:
joined.sort_values("bertscore_f1", ascending=False)[
    ["value_ai", "value_gt", "bertscore_f1"]
]

In [ ]:
row = joined.query("bertscore_f1 == bertscore_f1.max()").iloc[0]
print(row)
print(row["value_ai"])
print(row["value_gt"])
print(row["bertscore_f1"])

In [ ]:
row = joined.query("bertscore_f1 == bertscore_f1.min()").iloc[0]
print(row)
print(row["value_ai"])
print(row["value_gt"])
print(row["bertscore_f1"])